In [ ]:
from merfishing import Merfish
import numpy as np
from merfishing.tl.cellpose import run_cellpose
import pandas as pd
import h5py

## Entry Point

In [ ]:
#merfish = Merfish('dummy_experiment/output/region_0/')
merfish = Merfish('./output/region_0')


In [ ]:
# field of view id
fov =int(merfish.fov_ids[100])
# padding in pixel to add to the image boarder
padding = 150

In [ ]:
f = h5py.File('./output/region_0/detected_transcripts.hdf5', 'r')
assert len(set([key for key in f.keys()]) ^ set(merfish.fov_ids)) == 0

## Run Cellpose

In [ ]:
# merfish.cell_segmentation(model_type='nuclei', 
#                   diameter=100, 
#                   jobs=5, 
#                   padding=100,
#                   name='++DAPI',
#                   verbose=True, 
#                   redo=False,
#                  channels=[[3, 0]],
#                   debug =None)

## Run single cellpose

In [ ]:
from merfishing.tl.cellpose import run_cellpose
image = merfish.get_rgb_image('PolyT++DAPI', fov=fov, projection=None)

In [ ]:
# PolyT++DAPI means put PolyT in R channel (0) and DAPI in B channel (2)
# the image shape is (z, y, z, channel)
feature_mask, feature_meta = run_cellpose(image=image,
                      model_type='cyto',
                      diameter=100,
                      gpu=False,
                      channels=[[1, 3]],
                      channel_axis=3,
                      z_axis=0,
                      buffer_pixel_size=15,
                      plot=True)

In [ ]:
feature_mask, feature_meta = run_cellpose(image=image,
                                          pretrained_model_path = '/home/qz/project/merfish/hq_merfish/example/model/CP_20220803_144056',
                                          model_type='cyto',
                                          diameter=100,
                                          gpu=False,
                                          channels=[[1, 3]],
                                          channel_axis=3,
                                          z_axis=0,
                                          buffer_pixel_size=15,
                                          plot=True)

In [ ]:
# the image shape is (z, y, z, channel)
image = merfish.get_rgb_image('++DAPI', fov=fov, projection=None)
feature_mask, feature_meta = run_cellpose(image=image,
                      model_type='nuclei',
                      diameter=100,
                      gpu=False,
                      channels=[[3, 0]],
                      channel_axis=3,
                      z_axis=0,
                      buffer_pixel_size=15,
                      plot=True)

## Reopen Merfish Entry Point

Cellpose based segmentation will be used as default automatically

In [ ]:
merfish_new = Merfish('dummy_experiment/output/region_0/')

In [ ]:
fig = merfish_new.plot_fov(0, dpi=100)

In [ ]:
merfish_old = Merfish('dummy_experiment/output/region_0/', cell_segmentation='watershed')

In [ ]:
fig = merfish_old.plot_fov(0, dpi=100)

## Details - Cellpose2

Cell segmentation is done by the cellpose2 package with its pretrained model. Each z-plane is run separately, segmenting the PolyT channel with cyto model with the help of DAPI channel indicating nuclei. Please read the cellpose2 package for more details and fine tune the parameter if needed. You can train a custom model to achieve better segmentation results (watch the Youtube video below).

- Github: https://github.com/MouseLand/cellpose
- Documentation: https://cellpose.readthedocs.io/en/latest/
- YouTube Tutorial: https://www.youtube.com/watch?v=5qANHWoubZU


## Save TIFF file for Cellpose GUI and retrain the model

In [ ]:
import tifffile

name = 'PolyT++DAPI'  # R+G+B
fov = 0  # field of view
z = 3  # single z plane

img = merfish_new.get_rgb_image(name=name, z=z, fov=fov)

tifffile.imwrite(f'{fov}-{z}.tiff', img, compression='ZSTD')